In [ ]:
!pip install pandas scikit-learn deap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from deap import base, creator, tools, algorithms
import numpy as np

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataSets/News _dataset/Fake.csv')
# Assuming df has 'text' and 'label' columns
# For example purposes, we'll create a dummy dataset
data = {
    'text': ['Fake news article 1', 'Real news article 1', 'Fake news article 2', 'Real news article 2'],
    'label': [1, 0, 1, 0]
}
df = pd.DataFrame(data)

# Feature Extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text']).toarray()
y = df['label'].values

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Genetic Algorithm Configuration
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("indices", np.random.permutation, X_train.shape[1])
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_individual(individual):
    # Selecting features based on the individual mask
    mask = np.array(individual, dtype=bool)
    if np.sum(mask) == 0:  # Avoid empty feature set
        return 0,

    X_train_selected = X_train[:, mask]
    X_test_selected = X_test[:, mask]

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train_selected, y_train)
    y_pred = clf.predict(X_test_selected)

    accuracy = accuracy_score(y_test, y_pred)
    return accuracy,

toolbox.register("evaluate", eval_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Genetic Algorithm Execution
population = toolbox.population(n=10)
ngen = 5

for gen in range(ngen):
    # Evaluate the individuals
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.rand() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Replace the old population by the offspring
    population[:] = offspring

# Get the best individual
fits = [ind.fitness.values[0] for ind in population]
best_idx = np.argmax(fits)
best_individual = population[best_idx]

# Train the final model using the best features
best_mask = np.array(best_individual, dtype=bool)
X_train_best = X_train[:, best_mask]
X_test_best = X_test[:, best_mask]

final_model = RandomForestClassifier(random_state=42)
final_model.fit(X_train_best, y_train)
y_final_pred = final_model.predict(X_test_best)

# Evaluation
print("Final Model Accuracy:", accuracy_score(y_test, y_final_pred))
print(classification_report(y_test, y_final_pred))


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Final Model Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

